OK so now I do another attempt. The main thing however is that this time I'll try to augment the minority classes
so that they are more visible to the model.

The first part is exactly the same as before:

In [197]:
import torch
from torch import nn

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import FastICA
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import os

import pandas as pd

In [198]:
from tqdm.notebook import tqdm

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler


from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.datasets import load_iris

import math
import random

In [199]:
torch.manual_seed(0)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

In [200]:
###########################----CUstom Preprocessing function -----------####################

In [201]:
df = pd.read_csv("encoded_data.csv") #,header=None)

In [202]:
############################---- Preprocessing-----------------#######################

df.columns = [*df.columns[:-1], 'class']

np.random.seed(1)  
df = shuffle(df)
df.reset_index(drop=True)

train= df[0:round(len(df)*0.75)]
train = train.iloc[:,1:]
test=df [round(len(df)*0.75):len(df)]
test = test.iloc[:,1:]

#the .reset_index is to satisfy a quik of normalization e.g. minmaxscaler(). Not nescessary w/o
X = train.iloc[:, 0:-1].reset_index(drop=True)
y = train.iloc[:, -1].reset_index(drop=True)


In [203]:
###############-----------Normalize---------###################
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X.values)
X =pd.DataFrame(x_scaled,columns=X.columns)

y.reset_index(drop=True)
train = pd.concat([X,y],axis=1)
train

,"('Married',)","('Single',)","('Female',)","('Male',)","('Healthcare Representative',)","('Human Resources',)","('Laboratory Technician',)","('Manager',)","('Manufacturing Director',)","('Research Director',)",...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,class
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.000,0.250,0.667,0.000,0.250,0.167,0.000,0.471,1
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.333,1.000,0.000,0.000,0.000,0.000,1
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.175,0.333,0.667,0.125,0.111,0.000,0.059,1
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.333,0.025,0.333,0.667,0.025,0.056,0.000,0.000,0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.333,0.825,0.000,0.667,0.475,0.889,1.000,0.529,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.000,0.250,1.000,0.667,0.225,0.444,0.467,0.294,0
1098,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.300,0.333,1.000,0.200,0.167,0.000,0.412,0
1099,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.375,0.333,0.667,0.075,0.111,0.067,0.118,0
1100,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.000,0.250,0.333,0.333,0.250,0.222,0.067,0.471,0


In [126]:
######## Label Encoding ########

le = LabelEncoder()
le.fit(df.iloc[:,-1:])
df[4] = le.transform(df.iloc[:,-1:])
#I don't know why it deplicates the last column
df = df.iloc[:,:-1]
df.groupby('class')



/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Now this is where things differ, there are two data augmentation techniques that I used:

I first averaged 5 data points of class_1, and created a new data point of it, before adding it to our training dataset. 

In [204]:
############################----- DATA AUGMENT TECHNIQUE #1  -------###########################
# Don't execute this if you don't want to augment!

class_1 = train[train['class'] == 1]
class_1 = class_1.reset_index(drop=True)    

class_1['chunk'] = np.floor((class_1.index)/5)

extra_rows = class_1.groupby('chunk').mean()
class_1_augmented = class_1.drop('chunk',1)

train = pd.concat([train,class_1_augmented])

/tmp/ipykernel_7686/1554322261.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  class_1_augmented = class_1.drop('chunk',1)


In [ ]:
The second consisted of picking a point 

In [128]:
################## ---- DATA AUGMENT: GENERATE PTS ON RADIUS ######################
#http://extremelearning.com.au/how-to-generate-uniformly-random-points-on-n-spheres-and-n-balls/

AUGMENTED_RADIUS = 0.7

#################


    
def rand_n_sphere_pt(n):
    randoms = np.asarray([np.random.normal(0,1) for i in range(n)])
    d = (sum(randoms**2))**0.5

    return randoms/d

#n = number of pts you want created
def create_radius_aug(cumm_df, minority_class_train,  n):
    if n == 0:
        return pd.concat([minority_class_train,cumm_df])
    else:
        dim = len(minority_class_train.columns)

        aug_pts = minority_class_train.copy()
        aug_pts = aug_pts.apply(lambda x: x + rand_n_sphere_pt(dim)*AUGMENTED_RADIUS ,axis=1)
        aug_pts.iloc[:,-1:] = aug_pts.iloc[:,-1:].round()
    
        cumm_df = pd.concat([cumm_df,aug_pts])

        return create_radius_aug(cumm_df , minority_class_train,n-1)

empty_df = class_1_augmented[0:0]
    
train = pd.concat([train,create_radius_aug(empty_df,class_1_augmented,3)])
    

In [129]:
X = train.iloc[:, 0:-1]
y = train.iloc[:, -1]

#Normalize
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X.values)
X =pd.DataFrame(x_scaled)



In [130]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


In [131]:
df.sort_values(by='class')

,"('Divorced',)","('Married',)","('Single',)","('Female',)","('Male',)","('Healthcare Representative',)","('Human Resources',)","('Laboratory Technician',)","('Manager',)","('Manufacturing Director',)",...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,class
619,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,1,9,5,3,9,8,0,8,0
684,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,1,11,2,2,1,0,0,0,0
912,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,0,8,2,2,8,5,2,2,0
273,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,1,7,5,4,5,4,0,1,0
875,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,0,20,3,3,20,11,13,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,1,5,3,3,4,3,0,2,1
1058,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,80,0,16,2,4,15,9,10,10,1
946,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,0,9,2,3,5,4,1,0,1
1106,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,80,1,10,4,3,10,8,6,7,1


In [132]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cuda"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self,num_features,num_classes):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_features, num_features*2-1),
            nn.ReLU(),
            nn.Linear(num_features*2-1, num_features),
            nn.ReLU(),
            nn.Linear(num_features, num_classes),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


Using cuda device


In [133]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

accuracy_stats = {
                    'train': [],
                    "val": []}

loss_stats = {
                'train': [],
                "val": []}

In [134]:
###############------- CUSTOM LOSS FUNCTIONS ---------#########################################

class Monte_balanced_fn(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(Monte_balanced_fn, self).__init__()

    def forward(self, output, target, smooth=1): 
        
        #neural network output
        def add(loss , output, target, u_left):
            if u_left < 0:
                return loss
            else:

                #split NN output into two
                print(f"this is output:{output}")
                a1,a2 = output.split(split_size = 1,dim=1)
                pred = (a2-a1).floor() + 1

                pred_target = torch.column_stack((pred,target))


                #for each values of output_target that are == to indexed member of unique
                class_rows = pred_target[(pred_target[:, 1] == target.unique()[u_left])]
                pred,class_target = class_rows.split(1,dim=1)

                percentage_class_correct =torch.sum(1-(pred-class_target))/len(class_target)
                proportion_class_k = len(class_rows)/len(target)

                return add(loss + percentage_class_correct/proportion_class_k,output,target,u_left-1)

        u_left = len(target.unique())-1

        
        return add(0, output, target, u_left)



#########---- Geometric mean ----#######

class Geometric_mean(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(Geometric_mean, self).__init__()

    def forward(self, output, target, smooth=1): 
        
        #neural network output
        def add(loss , output, target, u_left , no_of_uniques):
            if u_left < 0:
                return loss**(1/no_of_uniques)
            else:
                #split NN output into two
                print(f"this is output:{output}")
                a1,a2 = output.split(split_size = 1,dim=1)
                pred = (a2-a1).floor() + 1

                pred_target = torch.column_stack((pred,target))


                #for each values of output_target that are == to indexed member of unique
                class_rows = pred_target[(pred_target[:, 1] == target.unique()[u_left])]
                pred,class_target = class_rows.split(1,dim=1)

                percentage_class_correct =torch.sum(1-(pred-class_target))/len(class_target)

                return add(loss*percentage_class_correct,output,target,u_left-1,no_of_uniques)

        no_of_unique = len(target.unique())
        #-1 is here because array starts with 0, while len() returns a normal count
        u_left = no_of_unique-1
        
        return add(0, output, target, u_left, no_of_unique)


In [135]:
################################------ Making NEURAL NETWORKS -------###########################################

def make_nn(df , verbose):
    print("Making Neural Network")
    
    X = df.iloc[:, 0:-1]
    y = df.iloc[:, -1]
        
        
#############--- PARAMETERS ------#####################

    NUM_FEATURES = len(X.columns)
    NUM_CLASSES = len(np.unique(y.values))
    LEARNING_RATE = 0.0007
    EPOCHS = 50
    BATCH_SIZE = 16
    
    OBJECTIVE_FUNCTION =nn.CrossEntropyLoss() #Geometric_mean() Monte_balanced_fn()  #

######################################################
    
    y = np.ravel(y)
    le = LabelEncoder()
    le.fit(y)
    y=le.transform(y)
        
    nn_error= False

    try:
        # Split into train+val and test
        X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, 
                                                                  #If class if imbalanced, use stratify!
                                                                  #stratify=y,
                                                                  random_state=69)

        # Split train into train-val
        X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1,
                                                          #stratify=y_trainval,
                                                          random_state=21)
    except Exception as e:
        print(e)
        nn_error= True

    if nn_error == True:
        return False

    #Normalize everything
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_val, y_val = np.array(X_val), np.array(y_val)
    X_test, y_test = np.array(X_test), np.array(y_test)


    train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
    val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
    test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
    
    
    train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          #sampler=weighted_sampler <--- In this example the classes are balanced, no need for weighted sampler
                         )
    val_loader = DataLoader(dataset=val_dataset, batch_size=1)
    test_loader = DataLoader(dataset=test_dataset, batch_size=1)

    model = NeuralNetwork(NUM_FEATURES,NUM_CLASSES).to(device)

    #loss function
    criterion = OBJECTIVE_FUNCTION #nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    accuracy_stats = {
                        'train': [],
                        "val": []}

    loss_stats = {
                    'train': [],
                    "val": []}

    print("Begin training.")

    for e in tqdm(range(1, EPOCHS+1)):

        # TRAINING
        train_epoch_loss = 0
        train_epoch_acc = 0

        model.train()
        for X_train_batch, y_train_batch in train_loader:
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
            optimizer.zero_grad()

            y_train_pred = model(X_train_batch)

            train_loss = criterion(y_train_pred, y_train_batch)
            train_acc = multi_acc(y_train_pred, y_train_batch)

            train_loss.backward()
            optimizer.step()

            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()


        # VALIDATION    
        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_acc = 0

            model.eval()
            for X_val_batch, y_val_batch in val_loader:
                #print(X_val_batch)
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred = model(X_val_batch)
                #print(y_val_pred)

                val_loss = criterion(y_val_pred, y_val_batch)
                val_acc = multi_acc(y_val_pred, y_val_batch)

                val_epoch_loss += val_loss.item()
                val_epoch_acc += val_acc.item()
                loss_stats['train'].append(train_epoch_loss/len(train_loader))

                loss_stats['val'].append(val_epoch_loss/len(val_loader))
                accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
                accuracy_stats['val'].append(val_epoch_acc/len(val_loader))

        if verbose:
            print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} %| Train Acc: {train_epoch_acc/len(train_loader):.3f} %| Val Acc: {val_epoch_acc/len(val_loader):.3f}')
    
    print(f"Final Val. Accuracy:{accuracy_stats['val'][-1]}")
    return model


In [179]:

############ TREE PARAMETERS ###################

KNN_PARAM = 11
HOMO_THRESHOLD = 0.7


In [180]:

################################################################
#Tree class
################################################################

class Tree:
    def __init__(self,name,df,homo_threshold):
        self.name = name
        self.df = df
        self.homo_threshold = homo_threshold
        self.X = df.iloc[:, 0:-1]
        self.y = df.iloc[:, -1]
        self.KNN = KNeighborsClassifier(n_neighbors=KNN_PARAM).fit(self.X,self.y)
        self.pred = self.KNN.predict(self.X)
        
        # if self.homo == 0 then KNN failed and you should self destruct it
        self.homo = self.homogeneity(self.pred, df)
        self.error = self.Error(self.pred,self.y)
        self.cluster_names = np.unique(self.pred)
        self.kids = self.create_kids() if self.homo <= homo_threshold else None
        self.kid_rank = None

        self.model = make_nn(df,verbose=False) if self.kids == None else None
        
    def Error(self,pred,y):
        correct= np.where(pred==y,True,False)
        perc = correct.sum()/len(correct)
        return perc
    
    def homogeneity(self, pred, y):
        #most_pop_class = self.pred.value_counts() <----this is good for dfs
        most_pop_class = max(np.unique(pred, return_counts = True)[1])
        homo = most_pop_class/len(y)
        return homo
        
    def check_thresh(self,cluster_name, X , y ,pred , homo_threshold):
        new_name = self.name + str(cluster_name) + "-"
        
        print(f"------------{new_name}----------")
        df_ = X.copy()
        df_['cluster'] = pred
        df_['class'] = y
        df_ = df_[df_['cluster']== cluster_name].drop('cluster',1)
        
        try:
            new_node = Tree(new_name,
                            df_,
                           homo_threshold)
            new_node.kid_rank=cluster_name 
            
        except Exception as e:
            print(e)
            new_node = None

        return new_node

    def create_kids(self):
        kids = [self.check_thresh(i, self.X, self.y , self.pred, self.homo_threshold) for i in self.cluster_names]
        #filter out "None/Ghost Kids"
        kids = list(filter(lambda kid: (kid != None ), kids))
        return kids
    
        #################################################
        
    def get_kids(self):
        for kid in self.kids:
            print(kid.kid_rank)

    
    def pachinko_test(self,x):
        if self.kids == None:
            print("No kids! Checking NN!")
            
            #Makes sure everything starts from 0 --- pytorch hates it when it doesn't
            y = np.ravel(self.y)
            le = LabelEncoder()
            le.fit(self.y)
            y=le.transform(self.y)
            
            with torch.no_grad():
                x = torch.Tensor([x]).to(device)
                y_pred = self.model(x)

                y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
                _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
                
            y_pred_tags = le.inverse_transform(y_pred_tags.to("cpu"))
            print(f"NN pred:{y_pred_tags}")

            
            return y_pred_tags
        else:
            indy_pred = self.KNN.predict([x])[0]
            print(f"Pachinko down we go!:{indy_pred}")
            
            chosen_one = list(filter(lambda kid: (kid.kid_rank == indy_pred ), self.kids))
            
            if chosen_one == []:
                return indy_pred
            else:
                return chosen_one[0].pachinko_test(x)
            
            
        
#################--- CREATE DA TREE!!!! --------#################################


root = Tree("root-",train,
            HOMO_THRESHOLD)


------------root-0.0-----------
Making Neural Network
Begin training.


/tmp/ipykernel_7686/3538571417.py:42: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_ = df_[df_['cluster']== cluster_name].drop('cluster',1)


  0%|          | 0/50 [00:00<?, ?it/s]

Final Val. Accuracy:100.0
------------root-1.0-----------
Making Neural Network
Begin training.


/tmp/ipykernel_7686/3538571417.py:42: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_ = df_[df_['cluster']== cluster_name].drop('cluster',1)


  0%|          | 0/50 [00:00<?, ?it/s]

Final Val. Accuracy:85.57692307692308


In [182]:
X = test.iloc[:, 0:-1]
y = test.iloc[:, -1]

y_preds = [int(root.pachinko_test(x)) for x in test.iloc[:,0:-1].values]

/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.

Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:1.0
No kids! Checking NN!
NN pred:[1.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:1.0
No kids! Checking NN!
NN pred:[1.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:1.0
No kids! Checking NN!
NN pred:[1.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:1.0
No kids! Checking NN!
NN pred:[1.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:0.0
No kids! Checking NN!
NN pred:[0.]
Pachinko down we go!:1.0
No kids! Checking NN!
NN pred:[1.]
Pachinko down we go!:0.0
No kids! Checki

/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/flackjacket/miniconda3/envs/machine-learning-env/lib/python3.9/site-packages/sklearn/base.

In [186]:
confusion_matrix(y,y_preds)

array([[228,  83],
       [ 35,  22]])

In [ ]:
So 

https://medium.com/@kohlishivam5522/understanding-a-classification-report-for-your-machine-learning-model-88815e2ce397

There are four ways to check if the predictions are right or wrong:

    TN / True Negative: the case was negative and predicted negative
    TP / True Positive: the case was positive and predicted positive
    FN / False Negative: the case was positive but predicted negative
    FP / False Positive: the case was negative but predicted positive
        
Precision = TP/(TP + FP) — What percent of your predictions were correct? 

Recall — What percent of the positive cases did you catch? Fraction of positives that were correctly identified.
Recall = TP/(TP+FN)

F1 score = 2*(Recall * Precision) / (Recall + Precision) — What percent of positive predictions were correct?

The F1 score is a weighted harmonic mean of precision and recall such that the best score is 1.0 and the worst is 0.0. F1 scores are lower than accuracy measures as they embed precision and recall into their computation. As a rule of thumb, the weighted average of F1 should be used to compare classifier models, not global accuracy.

Support -Support is the number of actual occurrences of the class in the specified dataset. 

In [144]:
print(classification_report(y,y_preds))

              precision    recall  f1-score   support

           0       0.87      0.74      0.80       311
           1       0.22      0.40      0.28        57

    accuracy                           0.68       368
   macro avg       0.54      0.57      0.54       368
weighted avg       0.77      0.68      0.72       368



So in conclusion, it is now better and worse in some ways. 
It misclassified a lot of class 0 and less of class 1. 


So... it is what it is for now. I'm not really sure if its really wise to keep banging on this dataset if no one benefits from it but there is some follow up work that can be done:

0)Further compartimentalize the dataset: e.g. run one model for each department
1)Explore different architectures (low priority)
2)Take a further look at the data
3)Explore dimension reduction and most importantly:
4)do further feature engineering

